In [ ]:
"""
MLP 기반 다중 레이블 이상 탐지 시스템 - 학습 스크립트
각 시나리오별 특징 벡터 파일을 참조하여 모델 학습
"""

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
from datetime import datetime

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# =====================================
# 설정
# =====================================
RANDOM_SEED = 42
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.001

# 데이터 파일 경로 (시나리오별)
DATA_FILES = {
    'normal': 'data/processed/1124_features.csv',
    'fluctuating': 'data/processed/1120_features_S1.csv',
    'unbalance': 'data/processed/1120_features_S2.csv'
}

# 레이블 매핑 (one-hot encoding)
LABEL_MAPPING = {
    'fluctuating': [1, 0],
    'unbalance': [0, 1],
    'normal': [0, 0]
}

# 출력 디렉토리 설정
OUTPUT_DIR_MODELS = 'models'
OUTPUT_DIR_RESULTS = 'results/mlp'

# 디렉토리 생성
os.makedirs(OUTPUT_DIR_MODELS, exist_ok=True)
os.makedirs(OUTPUT_DIR_RESULTS, exist_ok=True)

# 재현성 설정
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

# =====================================
# 1. PyTorch MLP 모델 정의
# =====================================
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_sizes=(64, 32), output_size=2):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], output_size)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# =====================================
# 2. 데이터 로드 및 레이블 생성
# =====================================
print("="*60)
print("📂 Loading Data from Multiple Scenarios")
print("="*60)

all_features = []
all_labels = []

for scenario, filepath in DATA_FILES.items():
    if not os.path.exists(filepath):
        print(f"⚠️  Warning: {filepath} not found, skipping {scenario}")
        continue
    
    # 특징 벡터 로드
    df = pd.read_csv(filepath)
    features = df.iloc[:, 3:].values  # window_id, start_time, end_time 제외
    
    # 레이블 생성
    label = np.array(LABEL_MAPPING[scenario])
    labels = np.tile(label, (len(features), 1))
    
    all_features.append(features)
    all_labels.append(labels)
    
    print(f"✅ Loaded {scenario:15s}: {features.shape[0]:4d} samples")

# 데이터 통합
X = np.vstack(all_features)
y = np.vstack(all_labels)

print(f"\n{'='*60}")
print(f"📊 Combined Dataset")
print(f"{'='*60}")
print(f"✅ Total samples: {X.shape[0]}")
print(f"✅ Feature shape: {X.shape}")
print(f"✅ Label shape: {y.shape}")

# 레이블 분포 확인
label_names = ['Fluctuating', 'unbalance', 'Normal']
for i, name in enumerate(label_names):
    count = np.sum(y[:, i] == 1) if i < 2 else np.sum(np.all(y == 0, axis=1))
    print(f"   - {name}: {count} samples")

# =====================================
# 3. 데이터 전처리 및 분할
# =====================================
print(f"\n{'='*60}")
print(f"🔧 Data Preprocessing")
print(f"{'='*60}")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.15, random_state=RANDOM_SEED, stratify=y.argmax(axis=1) if y.shape[1] > 1 else y
)

print(f"✅ Train samples: {X_train.shape[0]}")
print(f"✅ Test samples: {X_test.shape[0]}")

# =====================================
# 4. 스케일러 저장
# =====================================
scaler_path = os.path.join(OUTPUT_DIR_MODELS, 'scaler_mlp.pkl')
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"✅ Scaler saved: {scaler_path}")

# 훈련/테스트 데이터 저장 (예측 스크립트에서 사용)
data_split_path = os.path.join(OUTPUT_DIR_MODELS, 'data_split.npz')
np.savez(data_split_path, 
         X_train=X_train, X_test=X_test, 
         y_train=y_train, y_test=y_test)
print(f"✅ Data split saved: {data_split_path}")

# =====================================
# 5. PyTorch 데이터 준비
# =====================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n✅ Using device: {device}")

# NumPy to Tensor
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)

# DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# =====================================
# 6. 모델 초기화
# =====================================
input_size = X_train.shape[1]
model = MLPClassifier(input_size=input_size, hidden_sizes=(64, 32), output_size=2)
model = model.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"\n{'='*60}")
print(f"🏗️  Model Architecture")
print(f"{'='*60}")
print(f"   Input size: {input_size}")
print(f"   Architecture: {input_size} -> 64 -> 32 -> 2")
print(f"   Total parameters: {sum(p.numel() for p in model.parameters())}")

# =====================================
# 7. 모델 학습
# =====================================
print(f"\n{'='*60}")
print("🚀 Training MLP Classifier")
print(f"{'='*60}")

loss_history = []
start_time = datetime.now()

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    loss_history.append(avg_loss)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1:2d}/{EPOCHS}], Loss: {avg_loss:.4f}")

training_time = (datetime.now() - start_time).total_seconds()
print(f"\n✅ Training completed in {training_time:.2f} seconds")

# =====================================
# 8. 모델 저장
# =====================================
print(f"\n{'='*60}")
print("💾 Saving Models")
print(f"{'='*60}")

# PyTorch 모델 저장
model_path = os.path.join(OUTPUT_DIR_MODELS, 'mlp_classifier.pth')
torch.save({
    'model_state_dict': model.state_dict(),
    'input_size': input_size,
    'hidden_sizes': (64, 32),
    'output_size': 2,
    'label_names': label_names,
    'training_info': {
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'training_time': training_time,
        'final_loss': loss_history[-1],
        'train_samples': len(X_train),
        'test_samples': len(X_test),
        'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
}, model_path)
print(f"✅ PyTorch model saved: {model_path}")

# TorchScript 모델 저장 (최적화)
model.eval()
example_input = torch.randn(1, input_size).to(device)
traced_model = torch.jit.trace(model, example_input)
torchscript_path = os.path.join(OUTPUT_DIR_MODELS, 'mlp_classifier.pt')
traced_model.save(torchscript_path)
print(f"✅ TorchScript model saved: {torchscript_path}")

# =====================================
# 9. Loss Curve 시각화
# =====================================
print(f"\n{'='*60}")
print("📊 Generating Visualizations")
print(f"{'='*60}")

plt.figure(figsize=(10, 6))
plt.plot(loss_history, color='royalblue', linewidth=2, marker='o', markersize=4)
plt.title("MLP Training Loss Curve", fontsize=14, fontweight='bold')
plt.xlabel("Epochs", fontsize=12)
plt.ylabel("Loss (BCE)", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()

loss_curve_path = os.path.join(OUTPUT_DIR_RESULTS, 'mlp_loss_curve.png')
plt.savefig(loss_curve_path, dpi=300, bbox_inches='tight')
print(f"✅ Loss curve saved: {loss_curve_path}")
plt.close()

# =====================================
# 10. 훈련 정보 저장
# =====================================
training_summary = {
    'model_type': 'MLP Classifier (Multi-Label)',
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'data_sources': DATA_FILES,
    'label_mapping': LABEL_MAPPING,
    'dataset': {
        'total_samples': len(X),
        'train_samples': len(X_train),
        'test_samples': len(X_test),
        'feature_dim': input_size,
        'label_distribution': {
            label_names[i]: int(np.sum(y[:, i] == 1) if i < 2 else np.sum(np.all(y == 0, axis=1)))
            for i in range(len(label_names))
        }
    },
    'architecture': {
        'input_size': input_size,
        'hidden_layers': [64, 32],
        'output_size': 2,
        'total_parameters': sum(p.numel() for p in model.parameters())
    },
    'hyperparameters': {
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'optimizer': 'Adam',
        'loss_function': 'BCELoss',
        'random_seed': RANDOM_SEED
    },
    'training_results': {
        'training_time_seconds': training_time,
        'final_loss': float(loss_history[-1]),
        'min_loss': float(min(loss_history)),
        'loss_history': [float(l) for l in loss_history]
    },
    'saved_files': {
        'model': model_path,
        'torchscript': torchscript_path,
        'scaler': scaler_path,
        'data_split': data_split_path,
        'loss_curve': loss_curve_path
    }
}

summary_path = os.path.join(OUTPUT_DIR_MODELS, 'training_summary.json')
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(training_summary, f, indent=4, ensure_ascii=False)
print(f"✅ Training summary saved: {summary_path}")

# =====================================
# 결과 요약 출력
# =====================================
print(f"\n{'='*60}")
print("✅ Training Completed Successfully")
print(f"{'='*60}")
print(f"\n[Saved Models]")
print(f"  • PyTorch model: {model_path}")
print(f"  • TorchScript model: {torchscript_path}")
print(f"  • Scaler: {scaler_path}")
print(f"  • Data split: {data_split_path}")
print(f"\n[Training Summary]")
print(f"  • Training summary: {summary_path}")
print(f"  • Loss curve: {loss_curve_path}")
print(f"\n[Next Steps]")
print(f"  Run 'python predict_mlp.py' to evaluate and analyze the model")
print(f"{'='*60}\n")